# Jump process
Simon Frost (@sdwfrost), 2020-04-27

## Introduction

This implementation defines the model as a combination of two jump processes, infection and recovery, simulated using the [Doob-Gillespie algorithm](https://en.wikipedia.org/wiki/Gillespie_algorithm).

## Libraries

In [ ]:
using DifferentialEquations
using SimpleDiffEq
using Random
using DataFrames
using StatsPlots
using BenchmarkTools

## Transitions

For each process, we define the rate at which it occurs, and how the state variables change at each jump. Note that these are total rates, not *per capita*, and that the change in state variables occurs in-place.

In [ ]:
function infection_rate(u,p,t)
    (S,I,R) = u
    (β,c,γ) = p
    N = S+I+R
    β*c*I/N*S
end
function infection!(integrator)
  integrator.u[1] -= 1
  integrator.u[2] += 1
end
infection_jump = ConstantRateJump(infection_rate,infection!);

In [ ]:
function recovery_rate(u,p,t)
    (S,I,R) = u
    (β,c,γ) = p
    γ*I
end
function recovery!(integrator)
  integrator.u[2] -= 1
  integrator.u[3] += 1
end
recovery_jump = ConstantRateJump(recovery_rate,recovery!);

## Time domain

In [ ]:
tmax = 40.0
tspan = (0.0,tmax);

For plotting, we can also define a separate time series.

In [ ]:
δt = 0.1
t = 0:δt:tmax;

## Initial conditions

In [ ]:
u0 = [990,10,0]; # S,I,R

## Parameter values

In [ ]:
p = [0.05,10.0,0.25]; # β,c,γ

## Random number seed

We set a random number seed for reproducibility.

In [ ]:
Random.seed!(1234);

## Running the model

Running this model involves:

- Setting up the problem as a `DiscreteProblem`;
- Adding the jumps and setting the algorithm using `JumpProblem`; and
- Running the model, specifying `FunctionMap`

In [ ]:
#GE: What does DiscreteProblem do? How is it used? 
prob = DiscreteProblem(u0,tspan,p)

In [ ]:
#GE: How does JumpProblem work? 
prob_jump = JumpProblem(prob,Direct(),infection_jump,recovery_jump)

In [ ]:
sol_jump = solve(prob_jump,FunctionMap());
#sol_discrete = solve(prob_discrete) #, FunctionMap());
# sol_discrete is not correct

## Post-processing

In order to get output comparable across implementations, we output the model at a fixed set of times.

In [ ]:
out_jump = sol_jump(t);
#out_discrete = sol_discrete(t);

We can convert to a dataframe for convenience.

In [ ]:
df_jump = DataFrame(out_jump')
df_jump[!,:t] = out_jump.t;
println(size(df_jump), "  , ", size(df_jump))

#df_discrete = DataFrame(out_discrete')
#df_discrete[!,:t] = out_discrete.t;
#println(size(df_discrete), "  , ", size(df_discrete))

## Plotting

We can now plot the results.

In [ ]:
@df df_jump plot(:t,
    [:x1 :x2 :x3],
    label=["S" "I" "R"],
    xlabel="Time",
    ylabel="Number (jump)")

In [ ]:
"""
@df df_discrete plot(:t,
    [:x1 :x2 :x3],
    label=["S" "I" "R"],
    xlabel="Time",
    ylabel="Number (discrete)")
first(df_discrete, 10)
"""

## Benchmarking

In [ ]:
@benchmark solve(prob_jump,FunctionMap())